In [1]:
import xlwings as xw
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
xw.App.DisplayAlerts = False

# Current progress 6/23/2020
- Old format data has overall, fixed and ARM totals ready
- New format data has been renamed to match the old-format totals rows

# Stage 1: Data extraction functions

In [2]:
# Handles January and February 2007
def extract_jan07_feb07(month):
    report_loc = '/Users/Alex/Library/Group Containers/UBF8T346G9.Office/BSABS_2006-HE10_INVESTOR_REPORTS/' + \
        '2007 Through May 2013' + '/'
    filename = ''
    if month == 'jan':
        filename = report_loc + 'bear-stearns-2006-he10-investor-report-01-25-2007.xls'
    elif month == 'feb':
        filename = report_loc + 'bear-stearns-2006-he10-investor-report-02-26-2007.xls'
    investor_rpt = xw.Book(filename)
    LSS_part_one = investor_rpt.sheets['Loan Status Summary Part I']
    jan_or_feb = pd.DataFrame(columns=['Date','Current (Count)', 'Current (Balance)', 'Delinq 1 Month (Count)', \
    'Delinq 1 Month (Balance)', 'Delinq 2 Months (Count)', 'Delinq 2 Months (Balance)', 'Delinq 3+ Months (Count)', \
    'Delinq 3+ Months (Balance)', 'Bankruptcy (Count)', 'Bankruptcy (Balance)', 'Foreclosure (Count)', \
    'Foreclosure (Balance)','REO (Count)', 'REO (Balance)','Number of Loans'])
    locations = np.arange(30, 609, 34)
    for row_num in locations:
        row_loc = "A" + str(row_num) + ":O" + str(row_num)
        name_loc = "A" + str(row_num-1)
        current_data = np.concatenate((LSS_part_one.range(row_loc).options(np.array).value,\
            np.array(['NUM OF LOANS PLACEHOLDER'])))
        group_name = LSS_part_one.range(name_loc).value
        jan_or_feb.loc[group_name] = current_data
        curr_num = LSS_part_one.range('B' + str(row_num-17)).value
        one_delinq_num = LSS_part_one.range('D' + str(row_num-17)).value
        two_delinq_num = LSS_part_one.range('F' + str(row_num-17)).value
        three_plus_delinq_num = LSS_part_one.range('H' + str(row_num-17)).value
        bankruptcy_num = LSS_part_one.range('J' + str(row_num-17)).value
        foreclosure_num = LSS_part_one.range('L' + str(row_num-17)).value
        reo_num = LSS_part_one.range('N' + str(row_num-17)).value
        jan_or_feb.at[group_name,'Number of Loans'] = curr_num + one_delinq_num + two_delinq_num + \
            three_plus_delinq_num + bankruptcy_num + foreclosure_num + reo_num
    investor_rpt.close()
    jan_or_feb.index.rename('Group',inplace=True)
    return jan_or_feb

In [3]:
# Handles April 2007
def extract_apr07():
    report_loc = '/Users/Alex/Library/Group Containers/UBF8T346G9.Office/BSABS_2006-HE10_INVESTOR_REPORTS/' + \
        '2007 Through May 2013' + '/'
    filename = report_loc + 'bear-stearns-2006-he10-investor-report-04-25-2007.xls'
    investor_rpt = xw.Book(filename)
    LSS_part_one = investor_rpt.sheets['Loan Status Summary Part I']
    apr07 = pd.DataFrame(columns=['Date','Current (Count)', 'Current (Balance)', 'Delinq 1 Month (Count)', \
    'Delinq 1 Month (Balance)', 'Delinq 2 Months (Count)', 'Delinq 2 Months (Balance)', 'Delinq 3+ Months (Count)', \
    'Delinq 3+ Months (Balance)', 'Bankruptcy (Count)', 'Bankruptcy (Balance)', 'Foreclosure (Count)', \
    'Foreclosure (Balance)','REO (Count)', 'REO (Balance)','Number of Loans'])
    locations = np.arange(11, 281, 15)
    for row_num in locations:
        row_loc = "A" + str(row_num) + ":O" + str(row_num)
        name_loc = "A" + str(row_num-1)
        current_data = np.concatenate((LSS_part_one.range(row_loc).options(np.array).value,\
            np.array(['NUM OF LOANS PLACEHOLDER'])))
        group_name = LSS_part_one.range(name_loc).value
        apr07.loc[group_name] = current_data
        
        curr_num = LSS_part_one.range('B' + str(row_num-6)).options(np.array).value   
        one_delinq_num = LSS_part_one.range('D' + str(row_num-6)).options(np.array).value
        two_delinq_num = LSS_part_one.range('F' + str(row_num-6)).options(np.array).value
        three_plus_delinq_num = LSS_part_one.range('H' + str(row_num-6)).options(np.array).value
        bankruptcy_num = LSS_part_one.range('J' + str(row_num-6)).options(np.array).value
        foreclosure_num = LSS_part_one.range('L' + str(row_num-6)).options(np.array).value
        reo_num = LSS_part_one.range('N' + str(row_num-6)).options(np.array).value
        apr07.at[group_name,'Number of Loans'] = curr_num + one_delinq_num + two_delinq_num + \
            three_plus_delinq_num + bankruptcy_num + foreclosure_num + reo_num
    investor_rpt.close()
    apr07.index.rename('Group',inplace=True)
    return apr07

In [4]:
# The core data extraction function for January 2007 to May 2013
# This reads the LSS Part I sheet from every investor report in the given subfolder of my Office directory
def extract_jan07_may13():
    reports_loc = '/Users/Alex/Library/Group Containers/UBF8T346G9.Office/BSABS_2006-HE10_INVESTOR_REPORTS/' + \
        '2007 Through May 2013' + '/'
    reports_dir = os.listdir(reports_loc)
    reports_dir.sort(key=lambda z: z[-8:]+z[-14:-12])
    # Create a list to hold the monthly dataframes, which we'll concatenate at the end
    data_list = [x for x in range(len(reports_dir) - 1)]
    for i in range(len(reports_dir) - 1):
        filename = reports_loc + reports_dir[i]
        # Deal with the unique formats of the January, February and April 2007 reports
        if reports_dir[i] == 'bear-stearns-2006-he10-investor-report-01-25-2007.XLS':
            data_list[i] = extract_jan07_feb07('jan')
            continue
        elif reports_dir[i] == 'bear-stearns-2006-he10-investor-report-02-26-2007.XLS':
            data_list[i] = extract_jan07_feb07('feb')
            continue
        elif reports_dir[i] == 'bear-stearns-2006-he10-investor-report-04-25-2007.XLS':
            data_list[i] = extract_apr07()
            continue
        investor_rpt = xw.Book(filename)
        LSS_part_one = investor_rpt.sheets['Loan Status Summary Part I']
        # Set up a dataframe to hold an entire month's numbers
        one_month = pd.DataFrame(columns=['Date','Current (Count)', 'Current (Balance)', 'Delinq 1 Month (Count)', \
        'Delinq 1 Month (Balance)', 'Delinq 2 Months (Count)', 'Delinq 2 Months (Balance)', 'Delinq 3+ Months (Count)', \
        'Delinq 3+ Months (Balance)', 'Bankruptcy (Count)', 'Bankruptcy (Balance)', 'Foreclosure (Count)', \
        'Foreclosure (Balance)','REO (Count)', 'REO (Balance)','Number of Loans'])
        # The row numbers of the current-month percentage figures we're looking for
        locations = [30, 67, 104, 141, 178, 215, 252, 289, 326, 363, 400, 437, 474, 511, 548, 585, 622, 659]
        # For each row, grab all the data, and put it into the dataframe using the loan group name as the index value
        for row_num in locations:
            row_loc = "A" + str(row_num) + ":O" + str(row_num)
            name_loc = "A" + str(row_num-1) + ":A" + str(row_num-1)
            current_data = np.concatenate((LSS_part_one.range(row_loc).options(np.array).value,\
            np.array(['NUM OF LOANS PLACEHOLDER'])))
            group_name = LSS_part_one.range(name_loc).value
            one_month.loc[group_name] = current_data
            # Find the total number of mortgages still included in the deal that month
            curr_num = LSS_part_one.range('B' + str(row_num-17)).options(np.array).value   
            one_delinq_num = LSS_part_one.range('D' + str(row_num-17)).options(np.array).value
            two_delinq_num = LSS_part_one.range('F' + str(row_num-17)).options(np.array).value
            three_plus_delinq_num = LSS_part_one.range('H' + str(row_num-17)).options(np.array).value
            bankruptcy_num = LSS_part_one.range('J' + str(row_num-17)).options(np.array).value
            foreclosure_num = LSS_part_one.range('L' + str(row_num-17)).options(np.array).value
            reo_num = LSS_part_one.range('N' + str(row_num-17)).options(np.array).value
            one_month.at[group_name,'Number of Loans'] = curr_num + one_delinq_num + two_delinq_num + \
                three_plus_delinq_num + bankruptcy_num + foreclosure_num + reo_num
        # Put the current month's dataframe into the proper spot in the 'master list'
        data_list[i] = one_month
        investor_rpt.close()
        
    data_df = pd.concat(data_list)
    # Set up the MultiIndex of the final output dataframe
    data_df.index.rename('Group',inplace=True)
    data_df.set_index('Date',append=True,inplace=True)
    data_df = data_df.reorder_levels(['Date','Group'])
    return data_df

In [5]:
def extract_new_format_month(reports_loc, filename):
    full_path = reports_loc + filename
    investor_rpt = xw.Book(full_path)
    delinq_summary = investor_rpt.sheets['Delinquency Summary _ Lien and ']
    # Set up a dataframe to hold the entire month's numbers
    one_month = pd.DataFrame(columns=['Date','Current (Count)', 'Current (Balance)', 'Delinq 1 Month (Count)', \
        'Delinq 1 Month (Balance)', 'Delinq 2 Months (Count)', 'Delinq 2 Months (Balance)', 'Delinq 3+ Months (Count)', \
        'Delinq 3+ Months (Balance)', 'Bankruptcy (Count)', 'Bankruptcy (Balance)', 'Foreclosure (Count)', \
        'Foreclosure (Balance)','REO (Count)', 'REO (Balance)'],index=['Total - All Loans','Total - Fixed', \
        'Total - ARM'])
    
    # Find the correct sheet locations depending on the date of the investor report being accessed
    if int(filename[-8:-4]) < 2014 or (int(filename[-8:-4]) == 2014 and int(filename[-14:-12]) < 3):
        # June 2013 - February 2014
        # Create dictionaries of values to be extracted for each loan type and their locations in the investor report
        all_loans_dict = {'Current (Count)':'AB113','Delinq 1 Month (Count)':'L115','Delinq 2 Months (Count)':'P115', \
            'Delinq 3+ Months (Count)':['T115','X115'],'Bankruptcy (Count)':'AB116','Foreclosure (Count)': \
            'AB117','REO (Count)':'AB118'}
        # Find the total number of mortgages
        all_loans_denom = delinq_summary.range('AB119').value
        
        fixed_rate_dict = {'Current (Count)':'V146','Delinq 1 Month (Count)':'G147','Delinq 2 Months (Count)':'K147', \
            'Delinq 3+ Months (Count)':['O147','R147'],'Bankruptcy (Count)':'V148','Foreclosure (Count)': \
            'V149','REO (Count)':'V150'}
        # Find the total number of fixed-rate mortgages
        fixed_rate_denom = sum(delinq_summary.range('V146:V150').value)
        
        adj_rate_dict = {'Current (Count)':'V140','Delinq 1 Month (Count)':'G142','Delinq 2 Months (Count)':'K142', \
            'Delinq 3+ Months (Count)':['O142','R142'],'Bankruptcy (Count)':'V143','Foreclosure (Count)': \
            'V144','REO (Count)':'V145'}
        # Find the total number of adjustable-rate mortgages
        adj_rate_denom = sum(delinq_summary.range('V142:V145').value) + delinq_summary.range('V140').value
    elif int(filename[-8:-4]) < 2015 or (int(filename[-8:-4]) == 2015 and int(filename[-14:-12]) < 10):
        # March 2014 - September 2015
        all_loans_dict = {'Current (Count)':'AE109','Delinq 1 Month (Count)':'O111','Delinq 2 Months (Count)':'S111', \
            'Delinq 3+ Months (Count)':['W111','AA111'],'Bankruptcy (Count)':'AE112','Foreclosure (Count)': \
            'AE113','REO (Count)':'AE114'}
        all_loans_denom = delinq_summary.range('AE115').value
        fixed_rate_dict = {'Current (Count)':'AA142','Delinq 1 Month (Count)':'J143','Delinq 2 Months (Count)':'O143', \
            'Delinq 3+ Months (Count)':['S143','W143'],'Bankruptcy (Count)':'AA144','Foreclosure (Count)': \
            'AA145','REO (Count)':'AA146'}
        fixed_rate_denom = sum(delinq_summary.range('AA142:AA146').value)
        adj_rate_dict = {'Current (Count)':'AA136','Delinq 1 Month (Count)':'J138','Delinq 2 Months (Count)':'O138', \
            'Delinq 3+ Months (Count)':['S138','W138'],'Bankruptcy (Count)':'AA139','Foreclosure (Count)': \
            'AA140','REO (Count)':'AA141'}
        adj_rate_denom = sum(delinq_summary.range('AA138:AA141').value) + delinq_summary.range('AA136').value
    elif int(filename[-8:-4]) < 2016 or (int(filename[-8:-4]) == 2016 and int(filename[-14:-12]) < 5):
        # October 2015 - April 2016
        all_loans_dict = {'Current (Count)':'AG117','Delinq 1 Month (Count)':'P119','Delinq 2 Months (Count)':'U119', \
            'Delinq 3+ Months (Count)':['Y119','AC119'],'Bankruptcy (Count)':'AG120','Foreclosure (Count)': \
            'AG121','REO (Count)':'AG122'}
        all_loans_denom = delinq_summary.range('AG123').value
        fixed_rate_dict = {'Current (Count)':'AA154','Delinq 1 Month (Count)':'K155','Delinq 2 Months (Count)':'O155', \
            'Delinq 3+ Months (Count)':['S155','W155'],'Bankruptcy (Count)':'AA156','Foreclosure (Count)': \
            'AA157','REO (Count)':'AA158'}
        fixed_rate_denom = sum(delinq_summary.range('AA154:AA158').value)
        adj_rate_dict = {'Current (Count)':'AA147','Delinq 1 Month (Count)':'K150','Delinq 2 Months (Count)':'O150', \
            'Delinq 3+ Months (Count)':['S150','W150'],'Bankruptcy (Count)':'AA151','Foreclosure (Count)': \
            'AA152','REO (Count)':'AA153'}
        adj_rate_denom = sum(delinq_summary.range('AA150:AA153').value) + delinq_summary.range('AA147').value
    else:
        # May 2016 - March 2020
        all_loans_dict = {'Current (Count)':'AE117','Delinq 1 Month (Count)':'N119','Delinq 2 Months (Count)':'S119', \
            'Delinq 3+ Months (Count)':['W119','AA119'],'Bankruptcy (Count)':'AE120','Foreclosure (Count)': \
            'AE121','REO (Count)':'AE122'}
        all_loans_denom = delinq_summary.range('AE123').value
        fixed_rate_dict = {'Current (Count)':'AA154','Delinq 1 Month (Count)':'K155','Delinq 2 Months (Count)':'N155', \
            'Delinq 3+ Months (Count)':['S155','W155'],'Bankruptcy (Count)':'AA156','Foreclosure (Count)': \
            'AA157','REO (Count)':'AA158'}
        fixed_rate_denom = sum(delinq_summary.range('AA154:AA158').value)
        adj_rate_dict = {'Current (Count)':'AA147','Delinq 1 Month (Count)':'K150','Delinq 2 Months (Count)':'N150', \
            'Delinq 3+ Months (Count)':['S150','W150'],'Bankruptcy (Count)':'AA151','Foreclosure (Count)': \
            'AA152','REO (Count)':'AA153'}
        adj_rate_denom = sum(delinq_summary.range('AA150:AA153').value) + delinq_summary.range('AA147').value
    
    # Add on a column to track the total number of loans
    one_month['Number of Loans'] = 'ERROR' # If this doesn't get filled with a number, I'll be able to see that
    # Loop through all of the columns we want to retrieve data for and access their respective sheet locations
    for field in all_loans_dict.keys():
        if field == 'Delinq 3+ Months (Count)':
            months_3 = delinq_summary.range(all_loans_dict[field][0]).value
            months_4_or_more = delinq_summary.range(all_loans_dict[field][1]).value
            one_month.loc['Total - All Loans'][field] = (months_3 + months_4_or_more) / all_loans_denom
        else:
            one_month.loc['Total - All Loans'][field] = delinq_summary.range(all_loans_dict[field]).value \
                      / all_loans_denom
        one_month.loc['Total - All Loans']['Number of Loans'] = all_loans_denom
    # Perform the same task, but for the fixed-rate mortgages only
    for field in fixed_rate_dict.keys():
        if field == 'Delinq 3+ Months (Count)':
            months_3 = delinq_summary.range(fixed_rate_dict[field][0]).value
            months_4_or_more = delinq_summary.range(fixed_rate_dict[field][1]).value
            one_month.loc['Total - Fixed'][field] = (months_3 + months_4_or_more) / fixed_rate_denom
        else:
            one_month.loc['Total - Fixed'][field] = delinq_summary.range(fixed_rate_dict[field]).value \
                      / fixed_rate_denom
        one_month.loc['Total - Fixed']['Number of Loans'] = fixed_rate_denom
    # Get data one more time, but for the adjustable-rate mortgages only
    for field in adj_rate_dict.keys():
        if field == 'Delinq 3+ Months (Count)':
            months_3 = delinq_summary.range(adj_rate_dict[field][0]).value
            months_4_or_more = delinq_summary.range(adj_rate_dict[field][1]).value
            one_month.loc['Total - ARM'][field] = (months_3 + months_4_or_more) / adj_rate_denom
        else:
            one_month.loc['Total - ARM'][field] = delinq_summary.range(adj_rate_dict[field]).value \
                      / adj_rate_denom
        one_month.loc['Total - ARM']['Number of Loans'] = adj_rate_denom
    
    investor_rpt.close()
    one_month['Date'] = pd.to_datetime(filename[-14:-4])
    one_month.index.rename('Group',inplace=True)
    return one_month

In [6]:
def extract_new_format_all():
    reports_loc = '/Users/Alex/Library/Group Containers/UBF8T346G9.Office/BSABS_2006-HE10_INVESTOR_REPORTS/' + \
        'June 2013 Through March 2020' + '/'
    reports_dir = os.listdir(reports_loc)
    reports_dir.sort(key=lambda z: z[-8:]+z[-14:-12])
    # Create a list to hold the monthly dataframes, which we'll concatenate at the end
    data_list = [x for x in range(len(reports_dir) - 1)]
    for i in range(len(reports_dir) - 1):
        data_list[i] = extract_new_format_month(reports_loc, reports_dir[i])
    data_df = pd.concat(data_list)
    # Set up the MultiIndex of the final output dataframe
    data_df.set_index('Date',append=True,inplace=True)
    data_df = data_df.reorder_levels(['Date', 'Group'])
    return data_df

# Stage 2: Get the data

In [7]:
# Store all data from the old- and new-format extraction functions
old_format_all_data = extract_jan07_may13()
new_format_all_data = extract_new_format_all()

In [8]:
old_format_counts_only = old_format_all_data[['Current (Count)','Delinq 1 Month (Count)','Delinq 2 Months (Count)',\
    'Delinq 3+ Months (Count)','Bankruptcy (Count)','Foreclosure (Count)','REO (Count)', 'Number of Loans']]
old_format_counts_only

Current (Count)  \
Date       Group                                                         
2007-01-25 Total [All Loans]                                    0.9897   
           Group I Loans - Total                                0.9863   
           Group I Loans Fixed 1st Lien                         0.9891   
           Group I Loans Fixed 2nd Lien                         0.9519   
           Group I Loans 228 ARM                                0.9885   
...                                                                ...   
2013-05-28 Group II Loans Subgroup II 228 ARM                   0.5152   
           Group II Loans Subgroup II 327 ARM                   0.5122   
           Group II Loans Subgroup III Fixed 1st Lien           0.6198   
           Group II Loans Subgroup III 228 ARM                  0.4837   
           Group II Loans Subgroup III 327 ARM                  0.4706   

                                                       Delinq 1 Month (Count)  \
Date       Group                                                                
2007-01-25 Total [All Loans]                                           0.0103   
           Group I Loans - Total                                       0.0137   
           Group I Loans Fixed 1st Lien                                0.0109   
           Group I Loans Fixed 2nd Lien                                0.0481   
           Group I Loans 228 ARM                                       0.0115   
...                                                                       ...   
2013-05-28 Group II Loans Subgroup II 228 ARM                          0.0438   
           Group II Loans Subgroup II 327 ARM                          0.0244   
           Group II Loans Subgroup III Fixed 1st Lien                  0.0413   
           Group II Loans Subgroup III 228 ARM                         0.0272   
           Group II Loans Subgroup III 327 ARM                         0.0882   

                                                       Delinq 2 Months (Count)  \
Date       Group                                                                 
2007-01-25 Total [All Loans]                                            0.0000   
           Group I Loans - Total                                        0.0000   
           Group I Loans Fixed 1st Lien                                 0.0000   
           Group I Loans Fixed 2nd Lien                                 0.0000   
           Group I Loans 228 ARM                                        0.0000   
...                                                                        ...   
2013-05-28 Group II Loans Subgroup II 228 ARM                           0.0067   
           Group II Loans Subgroup II 327 ARM                           0.0244   
           Group II Loans Subgroup III Fixed 1st Lien                   0.0248   
           Group II Loans Subgroup III 228 ARM                          0.0380   
           Group II Loans Subgroup III 327 ARM                          0.0588   

                                                       Delinq 3+ Months (Count)  \
Date       Group                                                                  
2007-01-25 Total [All Loans]                                             0.0000   
           Group I Loans - Total                                         0.0000   
           Group I Loans Fixed 1st Lien                                  0.0000   
           Group I Loans Fixed 2nd Lien                                  0.0000   
           Group I Loans 228 ARM                                         0.0000   
...                                                                         ...   
2013-05-28 Group II Loans Subgroup II 228 ARM                            0.0741   
           Group II Loans Subgroup II 327 ARM                            0.0732   
           Group II Loans Subgroup III Fixed 1st Lien                    0.0826   
           Group II Loans Subgroup III 228 ARM            

# Stage 3: Clean the old-report-format dataframe
Requirements:
- Rename "Total \[All Loans\]"/"Total (All Loans)" to "All Loans"
- Take the weighted average of all fixed-rate loan data and record it as "Fixed"
- Take the weighted average of all ARM loan data and record it as "ARM"

NEW IDEA: Each month, create a new dataframe with that month's fixed data
Then, concatenate them all together at the end (like I do with data_list in the extraction functions)

NEWER IDEA: Create an entire new dataframe to hold the TOTAL, FIXED, and ARM old-report data

The idea behind this: Iterating by month, use old_format_counts_only to calculate the relevant metrics, but assign them to their respective rows in the NEW dataframe

In [9]:
# Skeleton of creating and filling in the Fixed and ARM rows
def set_up_totals(df, month):
    # For this particular month, set up rows to contain the total fixed and ARM data
    df.loc[(month,'Fixed'),:] = np.zeros((8))
    df.loc[(month,'ARM'),:] = np.zeros((8))
    fixed_only = df[df.index.get_level_values(1).str.contains('Fixed')]
    arm_only = df[df.index.get_level_values(1).str.contains('ARM')]
    # Save the quantities of fixed- and adjustable-rate loans for later
    fixed_count = fixed_only[fixed_only.index.get_level_values(0)==month]['Number of Loans'].sum()
    arm_count = arm_only[arm_only.index.get_level_values(0)==month]['Number of Loans'].sum()
    # Pull out the rows which are for fixed-rate loans, and those which are for adjustable-rate loans
    for col in df.columns.tolist()[:-1]:
        # For each column of data, find the weighted averages for fixed and adjustable rate loans
        fixed_weighted = fixed_only[fixed_only.index.get_level_values(0)==month][col] * \
            fixed_only[fixed_only.index.get_level_values(0)==month]['Number of Loans']
        arm_weighted = arm_only[arm_only.index.get_level_values(0)==month][col] * \
            arm_only[arm_only.index.get_level_values(0)==month]['Number of Loans']
        fixed_avg = fixed_weighted.sum() / \
            fixed_only[fixed_only.index.get_level_values(0)==month]['Number of Loans'].sum()
        arm_avg = arm_weighted.sum() / \
            arm_only[arm_only.index.get_level_values(0)==month]['Number of Loans'].sum()
        # Assign the weighted average to the general 'Fixed' and 'ARM' rows
        df.loc[(month,'Fixed'),col] = fixed_avg
        df.loc[(month,'ARM'),col] = arm_avg
    # Include the correct number of loans in the 'Fixed' and 'ARM' rows
    df.at[(month,'Fixed'),'Number of Loans'] = fixed_count
    df.at[(month,'ARM'),'Number of Loans'] = arm_count

In [10]:
# Process the old-format data to get totals rows for fixed and ARM loans
old_format_deep_copy = old_format_counts_only.copy(deep=True)
months = old_format_deep_copy.index.get_level_values(0)
months = months.drop_duplicates(keep='first')
for month in months:
    set_up_totals(old_format_deep_copy, month)
# Rename the rows in the new-format dataframe to match those from the old-format data
new_format_deep_copy = new_format_all_data.copy(deep=True)
new_format_groups = new_format_deep_copy.index.get_level_values(1).tolist()
for i in range(len(new_format_groups)):
    if new_format_groups[i] == 'Total - All Loans':
        new_format_groups[i] = 'Total [All Loans]'
    elif new_format_groups[i] == 'Total - Fixed':
        new_format_groups[i] = 'Fixed'
    elif new_format_groups[i] == 'Total - ARM':
        new_format_groups[i] = 'ARM'
new_format_dates = new_format_deep_copy.index.get_level_values(0)
new_format_deep_copy.index = pd.MultiIndex.from_arrays((new_format_dates,new_format_groups))
# Remove the Balance columns from the new-format data
new_format_final = new_format_deep_copy[['Current (Count)', 'Delinq 1 Month (Count)', \
                                'Delinq 2 Months (Count)', 'Delinq 3+ Months (Count)', 'Bankruptcy (Count)', \
                                'Foreclosure (Count)', 'REO (Count)', 'Number of Loans']]
new_format_final = new_format_final.sort_index(level=0)
# Remove the subgroup rows from the old-format data (but first clean the old-format data's Totals index names too)
old_format_groups = old_format_deep_copy.index.get_level_values(1).tolist()
for i in range(len(old_format_groups)):
    if old_format_groups[i] == 'Total (All Loans)':
        old_format_groups[i] = 'Total [All Loans]'
old_format_dates = old_format_deep_copy.index.get_level_values(0)
old_format_deep_copy.index = pd.MultiIndex.from_arrays((old_format_dates,old_format_groups))
old_format_total = old_format_deep_copy[old_format_deep_copy.index.get_level_values(1) == 'Total [All Loans]']
old_format_fixed = old_format_deep_copy[old_format_deep_copy.index.get_level_values(1) == 'Fixed']
old_format_arm = old_format_deep_copy[old_format_deep_copy.index.get_level_values(1) == 'ARM']
old_format_final = pd.concat([old_format_total,old_format_fixed,old_format_arm])
old_format_final = old_format_final.sort_index(level=0)

# Stage 4: Perform the final concatenation of old- and new-format data, and save the result

In [11]:
delinquency_stats_final = pd.concat([old_format_final, new_format_final])
with open('../../data/mbs_data_pickled/delinquency_stats.pkl','wb') as f:
    pickle.dump(delinquency_stats_final,f)

In [12]:
delinquency_stats_final

Current (Count) Delinq 1 Month (Count)  \
Date                                                                  
2007-01-25 ARM                      0.990543               0.009457   
           Fixed                    0.987984               0.012016   
           Total [All Loans]          0.9897                 0.0103   
2007-02-26 ARM                      0.970163               0.023682   
           Fixed                    0.979655               0.013159   
...                                      ...                    ...   
2020-02-25 Fixed                    0.842742               0.020161   
           Total [All Loans]        0.785981               0.029907   
2020-03-25 ARM                      0.751313               0.035026   
           Fixed                    0.839757               0.022312   
           Total [All Loans]        0.792293               0.029135   

                             Delinq 2 Months (Count) Delinq 3+ Months (Count)  \
Date                                                                            
2007-01-25 ARM                                   0.0                      0.0   
           Fixed                                 0.0                      0.0   
           Total [All Loans]                     0.0                      0.0   
2007-02-26 ARM                              0.005868                      0.0   
           Fixed                             0.00719                      0.0   
...                                              ...                      ...   
2020-02-25 Fixed                            0.016129                 0.010081   
           Total [All Loans]                0.014953                 0.017757   
2020-03-25 ARM                              0.008757                 0.029772   
           Fixed                            0.016227                 0.010142   
           Total [All Loans]                0.012218                 0.020677   

                             Bankruptcy (Count) Foreclosure (Count)  \
Date                                                                  
2007-01-25 ARM                              0.0                 0.0   
           Fixed                            0.0                 0.0   
           Total [All Loans]                0.0                 0.0   
2007-02-26 ARM                          0.00029                 0.0   
           Fixed                            0.0                 0.0   
...                                         ...                 ...   
2020-02-25 Fixed                       0.032258            0.066532   
           Total [All Loans]           0.057944            0.078505   
2020-03-25 ARM                         0.077058            0.082312   
           Fixed                       0.036511            0.058824   
           Total [All Loans]           0.058271            0.071429   

                             REO (Count) Number of Loans  
Date                                                      
2007-01-25 ARM                       0.0          3175.0  
           Fixed                     0.0          1580.0  
           Total [All Loans]         0.0          4759.0  
2007-02-26 ARM                       0.0          3416.0  
           Fixed                     0.0          1670.0  
...                                  ...             ...  
2020-02-25 Fixed                0.012097           496.0  
           Total [All Loans]    0.014953          1070.0  
2020-03-25 ARM                  0.015762           571.0  
           Fixed                0.016227           493.0  
           Total [All Loans]    0.015977          1064.0  

[477 rows x 8 columns]

## TEST CODE

In [13]:
new_format_all_data

Current (Count) Current (Balance)  \
Date       Group                                                 
2013-06-25 Total - All Loans        0.559372               NaN   
           Total - Fixed            0.674603               NaN   
           Total - ARM               0.49142               NaN   
2013-07-25 Total - All Loans        0.563772               NaN   
           Total - Fixed            0.686327               NaN   
...                                      ...               ...   
2020-02-25 Total - Fixed            0.842742               NaN   
           Total - ARM              0.736934               NaN   
2020-03-25 Total - All Loans        0.792293               NaN   
           Total - Fixed            0.839757               NaN   
           Total - ARM              0.751313               NaN   

                             Delinq 1 Month (Count) Delinq 1 Month (Balance)  \
Date       Group                                                               
2013-06-25 Total - All Loans               0.039254                      NaN   
           Total - Fixed                    0.03836                      NaN   
           Total - ARM                     0.039782                      NaN   
2013-07-25 Total - All Loans               0.040199                      NaN   
           Total - Fixed                   0.033512                      NaN   
...                                             ...                      ...   
2020-02-25 Total - Fixed                   0.020161                      NaN   
           Total - ARM                     0.038328                      NaN   
2020-03-25 Total - All Loans               0.029135                      NaN   
           Total - Fixed                   0.022312                      NaN   
           Total - ARM                     0.035026                      NaN   

                             Delinq 2 Months (Count)  \
Date       Group                                       
2013-06-25 Total - All Loans                0.019627   
           Total - Fixed                    0.019841   
           Total - ARM                      0.019501   
2013-07-25 Total - All Loans                 0.02134   
           Total - Fixed                    0.024129   
...                                              ...   
2020-02-25 Total - Fixed                    0.016129   
           Total - ARM                      0.013937   
2020-03-25 Total - All Loans                0.012218   
           Total - Fixed                    0.016227   
           Total - ARM                      0.008757   

                             Delinq 2 Months (Balance)  \
Date       Group                                         
2013-06-25 Total - All Loans                       NaN   
           Total - Fixed                           NaN   
           Total - ARM                             NaN   
2013-07-25 Total - All Loans                       NaN   
           Total - Fixed                           NaN   
...                                                ...   
2020-02-25 Total - Fixed                           NaN   
           Total - ARM                             NaN   
2020-03-25 Total - All Loans                       NaN   
           Total - Fixed                           NaN   
           Total - ARM                             NaN   

                             Delinq 3+ Months (Count)  \
Date       Group                                        
2013-06-25 Total - All Loans                 0.082924   
           Total - Fixed                     0.060847   
           Total - ARM                       0.095944   
2013-07-25 Total - All Loans                 0.120596   
           Total - Fixed                      0.08445   
...                                               ...   
2020-02-25 Total - Fixed                     0.010081   
           Total - ARM                        0.02439   
2020-03-25 Total - All Loans                 0.020677   
           Total -

In [14]:
next_test = old_format_counts_only.iloc[0:90].copy()
months = next_test.index.get_level_values(0)
months = months.drop_duplicates(keep='first')
for month in months:
    set_up_totals(next_test, month)
next_test.xs('2007-02-26',level=0)

,Current (Count),Delinq 1 Month (Count),Delinq 2 Months (Count),Delinq 3+ Months (Count),Bankruptcy (Count),Foreclosure (Count),REO (Count),Number of Loans
Group,,,,,,,,
Total [All Loans],0.973300,0.020200,0.006300,0.0,0.00020,0.0,0.0,5089.0
Group I Loans - Total,0.962700,0.027500,0.009200,0.0,0.00070,0.0,0.0,1529.0
Group I Loans Fixed 1st Lien,0.982400,0.014100,0.003500,0.0,0.00000,0.0,0.0,284.0
Group I Loans Fixed 2nd Lien,0.943400,0.018900,0.037700,0.0,0.00000,0.0,0.0,106.0
Group I Loans 228 ARM,0.960000,0.030900,0.008200,0.0,0.00090,0.0,0.0,1099.0
Group I Loans 327 ARM,0.950000,0.050000,0.000000,0.0,0.00000,0.0,0.0,40.0
Group II Loans - Total,0.977800,0.017100,0.005100,0.0,0.00000,0.0,0.0,3557.0
Group II Loans Subgroup I Fixed 1st Lien,0.992600,0.003700,0.003700,0.0,0.00000,0.0,0.0,270.0
Group II Loans Subgroup I Fixed 2nd Lien,0.964400,0.025400,0.010200,0.0,0.00000,0.0,0.0,393.0


In [15]:
jan_test = old_format_counts_only.xs('2007-01-25',level=0)
#jan_test.reset_index(inplace=True)

In [16]:
# Idea from https://songhuiming.github.io/pages/2017/03/18/weighted-avarage-aggrefated-function-with-apply-and-agg/
# This yields the average percent of fixed-rate loans which are current
fixed_curr = sum(fixed_only['Current (Count)'] * fixed_only['Number of Loans']) / fixed_only['Number of Loans'].sum()
arm_curr = sum(arm_only['Current (Count)'] * arm_only['Number of Loans']) / arm_only['Number of Loans'].sum()
print(fixed_curr, arm_curr)

NameError: name 'fixed_only' is not defined

In [ ]:
new_format_all_data

Current (Count) Current (Balance)  \
Date       Group                                                 
2013-06-25 Total - All Loans        0.559372               NaN   
           Total - Fixed            0.674603               NaN   
           Total - ARM               0.49142               NaN   
2013-07-25 Total - All Loans        0.563772               NaN   
           Total - Fixed            0.686327               NaN   
...                                      ...               ...   
2020-02-25 Total - Fixed            0.842742               NaN   
           Total - ARM              0.736934               NaN   
2020-03-25 Total - All Loans        0.792293               NaN   
           Total - Fixed            0.839757               NaN   
           Total - ARM              0.751313               NaN   

                             Delinq 1 Month (Count) Delinq 1 Month (Balance)  \
Date       Group                                                               
2013-06-25 Total - All Loans              0.0392542                      NaN   
           Total - Fixed                  0.0383598                      NaN   
           Total - ARM                    0.0397816                      NaN   
2013-07-25 Total - All Loans              0.0401985                      NaN   
           Total - Fixed                  0.0335121                      NaN   
...                                             ...                      ...   
2020-02-25 Total - Fixed                  0.0201613                      NaN   
           Total - ARM                    0.0383275                      NaN   
2020-03-25 Total - All Loans              0.0291353                      NaN   
           Total - Fixed                  0.0223124                      NaN   
           Total - ARM                    0.0350263                      NaN   

                             Delinq 2 Months (Count)  \
Date       Group                                       
2013-06-25 Total - All Loans               0.0196271   
           Total - Fixed                   0.0198413   
           Total - ARM                     0.0195008   
2013-07-25 Total - All Loans                 0.02134   
           Total - Fixed                   0.0241287   
...                                              ...   
2020-02-25 Total - Fixed                    0.016129   
           Total - ARM                     0.0139373   
2020-03-25 Total - All Loans                0.012218   
           Total - Fixed                   0.0162272   
           Total - ARM                    0.00875657   

                             Delinq 2 Months (Balance)  \
Date       Group                                         
2013-06-25 Total - All Loans                       NaN   
           Total - Fixed                           NaN   
           Total - ARM                             NaN   
2013-07-25 Total - All Loans                       NaN   
           Total - Fixed                           NaN   
...                                                ...   
2020-02-25 Total - Fixed                           NaN   
           Total - ARM                             NaN   
2020-03-25 Total - All Loans                       NaN   
           Total - Fixed                           NaN   
           Total - ARM                             NaN   

                             Delinq 3+ Months (Count)  \
Date       Group                                        
2013-06-25 Total - All Loans                0.0829244   
           Total - Fixed                    0.0608466   
           Total - ARM                      0.0959438   
2013-07-25 Total - All Loans                 0.120596   
           Total - Fixed                    0.0844504   
...                                               ...   
2020-02-25 Total - Fixed                    0.0100806   
           Total - ARM                      0.0243902   
2020-03-25 Total - All Loans                0.0206767   
           Total -

In [ ]:
# Test the new-format one-month extraction function
extract_new_format_month('/Users/Alex/Library/Group Containers/UBF8T346G9.Office/BSABS-HE10_INVESTOR_REPORTS/' + \
    'June 2013 Through March 2020/', 'bear-stearns-2006-he10-investor-report-11-25-2016.xls')

,Date,Current (Count),Current (Balance),Delinq 1 Month (Count),Delinq 1 Month (Balance),Delinq 2 Months (Count),Delinq 2 Months (Balance),Delinq 3+ Months (Count),Delinq 3+ Months (Balance),Bankruptcy (Count),Bankruptcy (Balance),Foreclosure (Count),Foreclosure (Balance),REO (Count),REO (Balance),Number of Loans
Group,,,,,,,,,,,,,,,,
Total - All Loans,2016-11-25,0.673469,NaN,0.0333333,NaN,0.0163265,NaN,0.0741497,NaN,0.0408163,NaN,0.136735,NaN,0.0251701,NaN,1470
Total - Fixed,2016-11-25,0.764228,NaN,0.0325203,NaN,0.0211382,NaN,0.0617886,NaN,0.0292683,NaN,0.0796748,NaN,0.0113821,NaN,615
Total - ARM,2016-11-25,0.608187,NaN,0.0339181,NaN,0.0128655,NaN,0.0830409,NaN,0.0491228,NaN,0.177778,NaN,0.0350877,NaN,855


In [ ]:
# Old test of the old-format extraction function
extract_jan07_may13('2007 Through May 2013')

Current (Count)  \
Date       Group                                                         
2007-01-25 Total [All Loans]                                    0.9897   
           Group I Loans - Total                                0.9863   
           Group I Loans Fixed 1st Lien                         0.9891   
           Group I Loans Fixed 2nd Lien                         0.9519   
           Group I Loans 228 ARM                                0.9885   
...                                                                ...   
2007-06-25 Group II Loans Subgroup II 228 ARM                   0.9095   
           Group II Loans Subgroup II 327 ARM                   0.9474   
           Group II Loans Subgroup III Fixed 1st Lien           0.9495   
           Group II Loans Subgroup III 228 ARM                  0.8780   
           Group II Loans Subgroup III 327 ARM                  0.8864   

                                                       Current (Balance)  \
Date       Group                                                           
2007-01-25 Total [All Loans]                                      0.9890   
           Group I Loans - Total                                  0.9859   
           Group I Loans Fixed 1st Lien                           0.9890   
           Group I Loans Fixed 2nd Lien                           0.9443   
           Group I Loans 228 ARM                                  0.9858   
...                                                                  ...   
2007-06-25 Group II Loans Subgroup II 228 ARM                     0.8977   
           Group II Loans Subgroup II 327 ARM                     0.9646   
           Group II Loans Subgroup III Fixed 1st Lien             0.9604   
           Group II Loans Subgroup III 228 ARM                    0.8864   
           Group II Loans Subgroup III 327 ARM                    0.8986   

                                                       Delinq 1 Month (Count)  \
Date       Group                                                                
2007-01-25 Total [All Loans]                                           0.0103   
           Group I Loans - Total                                       0.0137   
           Group I Loans Fixed 1st Lien                                0.0109   
           Group I Loans Fixed 2nd Lien                                0.0481   
           Group I Loans 228 ARM                                       0.0115   
...                                                                       ...   
2007-06-25 Group II Loans Subgroup II 228 ARM                          0.0401   
           Group II Loans Subgroup II 327 ARM                          0.0316   
           Group II Loans Subgroup III Fixed 1st Lien                  0.0152   
           Group II Loans Subgroup III 228 ARM                         0.0820   
           Group II Loans Subgroup III 327 ARM                         0.0227   

                                                       Delinq 1 Month (Balance)  \
Date       Group                                                                  
2007-01-25 Total [All Loans]                                             0.0110   
           Group I Loans - Total                                         0.0141   
           Group I Loans Fixed 1st Lien                                  0.0110   
           Group I Loans Fixed 2nd Lien                                  0.0557   
           Group I Loans 228 ARM                                         0.0142   
...                                                                         ...   
2007-06-25 Group II Loans Subgroup II 228 ARM                            0.0427   
           Group II Loans Subgroup II 327 ARM                            0.0133   
           Group II Loans Subgroup III Fixed 1st Lien                    0.0075   
           Group II Loans Subgroup III 228 ARM                           0.0722   
           Group II Loans Subgroup III 327 ARM       

In [ ]:
jan_test[jan_test.Group.str.contains('Fixed')]['Current (Count)'].multiply(jan_test[jan_test.Group.str.contains('Fixed')]['Number of Loans'] / fixed_num).sum()

0.9879837974683544

In [ ]:
# OLD IDEA
fixed_num = jan_test[jan_test.Group.str.contains('Fixed')]['Number of Loans'].sum()
fixed_only = jan_test[jan_test.Group.str.contains('Fixed')]
arm_num = jan_test[jan_test.Group.str.contains('ARM')]['Number of Loans'].sum()
arm_only = jan_test[jan_test.Group.str.contains('ARM')]

## Idea for New Format structure
- get_data function allocates items to the data_list and concatenates it at the end
- get_data calls extract_one_month and passes in the filename, and extract_one_month returns the data for that month
- extract_one_month chooses which row-location dictionaries to use based on the date in the filename